# Table of contents
1. [Libraries](#libraries)
2. [Numerical columns](#numerical)
3. [Categorical columns](#categorical)



<h1 id = "libraries"> 1. Libraries </h1>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import seaborn as sns
from scipy.stats import sem

from sklearn.impute import KNNImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from collections import Counter

In [2]:
raw_data = pd.read_csv('../Data/retyped_data.csv')

<div id = "numerical"> <h1> 2. Numerical columns </h1> </div>

- Impute missing values using KNN strategy

In [3]:
# Prepare the data (numeric columns only)
columns = ['Tomatoes CriticScore', 'Tomatoes UserScore', 'Metascore', 'Meta UserScore']

# Copy raw data and scale 'Meta UserScore'
data_tmp_2 = raw_data.copy()
data_tmp_2['Meta UserScore'] = data_tmp_2['Meta UserScore'] * 10

# Create an imputer
knn_imputer = KNNImputer(n_neighbors=20, weights="uniform")

# Step 1: Apply KNN Imputation
# Impute all missing values
imputed_data = knn_imputer.fit_transform(data_tmp_2[columns])

# Reconstruct the DataFrame with imputed values
imputed_df = pd.DataFrame(imputed_data, columns=columns)

# Step 2: Fill missing values only
for col in columns:
    # Update only rows where the value is NaN
    data_tmp_2[col] = data_tmp_2[col].combine_first(imputed_df[col])

data_tmp_2.isnull().sum()

Title                      0
Tomatoes CriticScore       0
Tomatoes UserScore         0
Link                       0
PlatformReleased           0
Cast                      32
Director                  35
Genre                     46
Rating                  1091
Runtime                   70
Studio                    47
Release Date              75
Production Budget          0
Domestic Gross             0
Worldwide Gross            0
Metascore                  0
Meta UserScore             0
dtype: int64

<div id = "categorical"> <h1> 3. Categorical columns </h1> </div>

In [4]:
columns = ['Cast', 'Director', 'Genre', 'Rating', 'Studio', 'Release Date']

<h3>Cast Column<h3>

In [5]:
# First, let's look at the distribution of Cast data
print("Number of missing values in Cast:", data_tmp_2['Cast'].isnull().sum())

# Create helper functions to process Cast
def process_cast(cast_string):
    """Normalize Cast string"""
    if pd.isna(cast_string):
        return []
    # Handle incorrect format cases
    cast_string = str(cast_string)
    cast_string = cast_string.replace('[', '').replace(']', '')  # Remove square brackets
    # Split actors and remove extra whitespace
    return [actor.strip() for actor in cast_string.split(',')]

def get_similar_movies_for_cast(row, data, n_similar=5):
    """Find similar movies based on Director and Genre"""
    # If both Director and Genre are missing, return random sample
    if pd.isna(row['Director']) and pd.isna(row['Genre']):
        return data.sample(n=n_similar)
    
    # Create initial mask for all rows
    mask = pd.Series(True, index=data.index)
    
    # Filter by Director if available
    if not pd.isna(row['Director']):
        mask &= (data['Director'] == row['Director'])
    # Filter by Genre if available  
    if not pd.isna(row['Genre']):
        mask &= (data['Genre'] == row['Genre'])
    
    # Get movies matching both filters
    similar_movies = data[mask]

    # If not enough similar movies, get more movies with same Director
    if len(similar_movies) < n_similar and not pd.isna(row['Director']):
        director_movies = data[data['Director'] == row['Director']]
        # Combine and remove duplicates
        similar_movies = pd.concat([similar_movies, director_movies]).drop_duplicates()
    
    # Return top n similar movies
    return similar_movies.head(n_similar)

def fill_cast(row, data):
    """Fill missing Cast based on similar movies"""
    if pd.isna(row['Cast']):
        similar_movies = get_similar_movies_for_cast(row, data)
        # Get all casts from similar movies
        all_casts = []
        for _, movie in similar_movies.iterrows():
            if not pd.isna(movie['Cast']):
                all_casts.extend(process_cast(movie['Cast']))
        
        # Get the most frequent actors
        if all_casts:
            most_common = Counter(all_casts).most_common(3)
            # Create string list with correct format
            actors = [actor for actor, _ in most_common]
            return f"[{', '.join(f'{actor}' for actor in actors)}]"
        return 'Unknown Cast'
    return row['Cast']

# Save index of rows with missing Cast before filling
missing_cast_idx = data_tmp_2[data_tmp_2['Cast'].isnull()].index

# Apply missing values filling
data_tmp_2['Cast'] = data_tmp_2.apply(lambda row: fill_cast(row, data_tmp_2), axis=1)

print("Number of missing values in Cast after Preprocess:", data_tmp_2['Cast'].isnull().sum())

Number of missing values in Cast: 32
Number of missing values in Cast after Preprocess: 0


<h3>Director Column<h3>

In [6]:
# First look at the distribution of Director
print("Number of missing values in Director:", data_tmp_2['Director'].isnull().sum())

def get_similar_movies_for_director(row, data, n_similar=5):
    """Find similar movies based on Cast and Genre"""
    if pd.isna(row['Cast']) and pd.isna(row['Genre']):
        return data.sample(n=n_similar)
    
    mask = pd.Series(True, index=data.index)
    
    # Use filled Cast
    if not pd.isna(row['Cast']):
        # Get list of actors
        row_cast = set(process_cast(row['Cast']))
        # Find movies with at least 1 matching actor
        cast_mask = data['Cast'].apply(lambda x: bool(row_cast.intersection(set(process_cast(x)))))
        mask &= cast_mask
    
    if not pd.isna(row['Genre']):
        mask &= (data['Genre'] == row['Genre'])
    
    similar_movies = data[mask]
    
    # If not enough similar movies, expand by Genre
    if len(similar_movies) < n_similar and not pd.isna(row['Genre']):
        genre_movies = data[data['Genre'] == row['Genre']]
        similar_movies = pd.concat([similar_movies, genre_movies]).drop_duplicates()
    
    return similar_movies.head(n_similar)

def fill_director(row, data):
    """Fill missing Director based on similar movies"""
    if pd.isna(row['Director']):
        similar_movies = get_similar_movies_for_director(row, data)
        # Get directors from similar movies
        directors = []
        for _, movie in similar_movies.iterrows():
            if not pd.isna(movie['Director']):
                directors.append(movie['Director'])
        
        # Get most frequent director
        if directors:
            most_common = Counter(directors).most_common(1)
            return most_common[0][0]
        return 'Unknown Director'
    return row['Director']

# Save indices of rows with missing Director
missing_director_idx = data_tmp_2[data_tmp_2['Director'].isnull()].index

# Apply fill_director
data_tmp_2['Director'] = data_tmp_2.apply(lambda row: fill_director(row, data_tmp_2), axis=1)

# Print number of missing values after filling
print("Number of missing values in Director after Preprocess:", data_tmp_2['Director'].isnull().sum())

Number of missing values in Director: 35
Number of missing values in Director after Preprocess: 0


<h3>Genre Column<h3>

In [7]:
# First, let's look at the distribution of Genre data
print("Number of missing values in Genre:", data_tmp_2['Genre'].isnull().sum())

# Create helper functions to process Genre
def process_genre(genre_string):
    """Normalize Genre string"""
    if pd.isna(genre_string):
        return []
    # Handle incorrect format cases
    genre_string = str(genre_string)
    genre_string = genre_string.replace('[', '').replace(']', '')  # Remove square brackets
    # Split genres and remove extra whitespace
    return [genre.strip() for genre in genre_string.split(',')]

def get_similar_movies_for_genre(row, data, n_similar=5):
    """Find similar movies based on Cast and Director"""
    if pd.isna(row['Cast']) and pd.isna(row['Director']):
        return data.sample(n=n_similar)
    
    mask = pd.Series(True, index=data.index)
    
    # Filter by Cast if available
    if not pd.isna(row['Cast']):
        mask &= (data['Cast'] == row['Cast'])
    
    # Filter by Director if available
    if not pd.isna(row['Director']):
        mask &= (data['Director'] == row['Director'])
    
    similar_movies = data[mask]
    
    # If not enough similar movies, get more movies with same Director
    if len(similar_movies) < n_similar and not pd.isna(row['Director']):
        director_movies = data[data['Director'] == row['Director']]
        similar_movies = pd.concat([similar_movies, director_movies]).drop_duplicates()
    
    return similar_movies.head(n_similar)


def fill_genre(row, data):
    """Fill missing Genre based on similar movies"""
    if pd.isna(row['Genre']):
        similar_movies = get_similar_movies_for_genre(row, data)
        # Get all genres from similar movies
        all_genres = []
        for _, movie in similar_movies.iterrows():
            if not pd.isna(movie['Genre']):
                all_genres.extend(process_genre(movie['Genre']))
        
        # Get the most frequent genres
        if all_genres:
            most_common = Counter(all_genres).most_common(2)  # Get 2 most common genres
            # Create string list with correct format
            genres = [genre for genre, _ in most_common]
            return f"[{', '.join(f'{genre}' for genre in genres)}]"
        return 'Unknown Genre'
    return row['Genre']

# Save index of rows with missing Genre before filling
missing_genre_idx = data_tmp_2[data_tmp_2['Genre'].isnull()].index

# Apply missing values filling
data_tmp_2['Genre'] = data_tmp_2.apply(lambda row: fill_genre(row, data_tmp_2), axis=1)

print("Number of missing values in Genre after Preprocess:", data_tmp_2['Genre'].isnull().sum())

Number of missing values in Genre: 46
Number of missing values in Genre after Preprocess: 0
